### Health and Nutrition Worldwide

This dataset was downloaded from kaggle on 7 feb 23 and this analysis has the following objectives:
- Understand how region and income group influence the health and nutrition indicators
- How the health and nutrition indicators are related between themselves
- Understand if we can group the countries based on the indicators provided
- Understand which zones have been improving and declining their indicators

In [51]:
# !kaggle datasets download -d sivamsinghsh/health-nutrition-and-population-statistics

In [52]:
# import zipfile

# with zipfile.ZipFile('health-nutrition-and-population-statistics.zip', 'r') as zip_ref:
#     zip_ref.extractall()

Analysing the Excel file we can see that there are three sheets we will need to work with:
- _Data_ : The whole dataset with the indicator values
- _Country_ : We will use the columns _Region_ and _Income Group_ from this dataset
- _Series_ : We will use the column _Topic_ for our analysis

In [53]:
import pandas as pd
import regex as re

In [54]:
indicators_dataset = pd.read_excel('HNP_StatsEXCEL.xlsx', sheet_name='Data')
country_dataset = pd.read_excel('HNP_StatsEXCEL.xlsx', sheet_name='Country')
series_dataset = pd.read_excel('HNP_StatsEXCEL.xlsx', sheet_name='Series')

Having our datasets imported, let's filter the relevant columns for our analysis

In [55]:
country_dataset = country_dataset[['Country Code','Region','Income Group']]
series_dataset = series_dataset[['Series Code','Topic']]

Let's take a look at our datasets now

In [56]:
indicators_dataset.head(2)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Africa Eastern and Southern,AFE,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,141.457567,141.603817,141.796749,141.651778,141.595374,141.593273,...,105.321998,103.629032,101.905042,100.133826,98.367869,96.574004,95.011793,93.43222,91.845198,NaN
1,Africa Eastern and Southern,AFE,Adults (ages 15+) and children (0-14 years) li...,SH.HIV.TOTL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
country_dataset.head(2)

,Country Code,Region,Income Group
0,ABW,Latin America & Caribbean,High income
1,AFE,NaN,NaN


In [58]:
series_dataset.head(10)

,Series Code,Topic
0,HD.HCI.OVRL,Public Sector: Policy & institutions
1,HD.HCI.OVRL.FE,Public Sector: Policy & institutions
2,HD.HCI.OVRL.LB,Public Sector: Policy & institutions
3,HD.HCI.OVRL.LB.FE,Public Sector: Policy & institutions
4,HD.HCI.OVRL.LB.MA,Public Sector: Policy & institutions
5,HD.HCI.OVRL.MA,Public Sector: Policy & institutions
6,HD.HCI.OVRL.UB,Public Sector: Policy & institutions
7,HD.HCI.OVRL.UB.FE,Public Sector: Policy & institutions
8,HD.HCI.OVRL.UB.MA,Public Sector: Policy & institutions
9,NY.GNP.PCAP.CD,Background: Economy


Right away we can see that the dataset is organized in a way which has a column for each year represented, which might not be suitable for our analysis. For now, let's reshape the dataframe, creating a single column for the year and a column for the value. We will also create a new dataset with only the most recent year, in order to assess and analyse the current state of the indicators.

Also, as a good pratice, let's rename the column names to lowercase and separated by an underscore

In [59]:
column_mapper = {
    'Country Name':'country_name',
    'Country Code':'country_code',
    'Indicator Name':'indicator_name',
    'Indicator Code':'series_code',
    'Region':'region',
    'Income Group':'income_group',
    'Series Code':'series_code',
    'Topic':'topic'
}

indicators_dataset = indicators_dataset.rename(columns=column_mapper)
country_dataset = country_dataset.rename(columns=column_mapper)
series_dataset = series_dataset.rename(columns=column_mapper)

In [60]:
indicators_dataset = indicators_dataset.melt(id_vars=['country_name','country_code','indicator_name','series_code'], var_name='year' )

In [61]:
indicators_dataset_2021 = indicators_dataset.loc[indicators_dataset['year']=='2021'].copy()

Looking at the _series_dataset_, we can also verify that the _Topic_ column seems to have a main category and a secondary category. It might become helpfull to have this categories separated.

In [62]:
series_dataset[['main_topic','secondary_topic']] = series_dataset['topic'].str.split(':',expand=True)

Let's start out analysis! Since we have a large number of countries and indicators represented, we will start by analysing the data by the top level categories (_Region_, _Income Group_, _Main Topic_) and drill down as we see fit. We will also start by 2021 only and later we will analyse the evolution of the indicators throughout time.

In [63]:
dataset_2021 = indicators_dataset_2021\
                .merge(country_dataset, how='left', on='country_code', suffixes=['_d','_c'])\
                .merge(series_dataset, how='left', on='series_code', suffixes=['_d','_s'])

In [64]:
dataset_2021.drop(['country_code','series_code'], axis=1, inplace=True)

In [65]:
dataset_2021.head(3)

,country_name,indicator_name,year,value,region,income_group,topic,main_topic,secondary_topic
0,Africa Eastern and Southern,"Adolescent fertility rate (births per 1,000 wo...",2021,NaN,NaN,NaN,Reproductive health,Reproductive health,None
1,Africa Eastern and Southern,Adults (ages 15+) and children (0-14 years) li...,2021,NaN,NaN,NaN,HIV/AIDS,HIV/AIDS,None
2,Africa Eastern and Southern,Adults (ages 15+) and children (ages 0-14) new...,2021,NaN,NaN,NaN,HIV/AIDS,HIV/AIDS,None


By looking at the result table we can see that we are missing some values in the columns _value_, _region_ and _income_group_. Although we cannot attribute any values to _value_ since these are missing due to the data not being collected, we can assign a category of 'No Information' to _income_group_. We can also verify if the countries without region aren't a region per se, like 'Africa Eastern and Southern'.

In [66]:
dataset_2021.loc[dataset_2021['income_group'].isna(), 'income_group'] = 'No Information'

In [67]:
dataset_2021.loc[dataset_2021['region'].isna(), 'country_name'].unique()

array(['Africa Eastern and Southern', 'Africa Western and Central',
       'Arab World', 'Caribbean small states',
       'Central Europe and the Baltics', 'Early-demographic dividend',
       'East Asia & Pacific',
       'East Asia & Pacific (excluding high income)',
       'East Asia & Pacific (IDA & IBRD countries)', 'Euro area',
       'Europe & Central Asia',
       'Europe & Central Asia (excluding high income)',
       'Europe & Central Asia (IDA & IBRD countries)', 'European Union',
       'Fragile and conflict affected situations',
       'Heavily indebted poor countries (HIPC)', 'High income',
       'IBRD only', 'IDA & IBRD total', 'IDA blend', 'IDA only',
       'IDA total', 'Late-demographic dividend',
       'Latin America & Caribbean',
       'Latin America & Caribbean (excluding high income)',
       'Latin America & the Caribbean (IDA & IBRD countries)',
       'Least developed countries: UN classification',
       'Low & middle income', 'Low income', 'Lower middle in

Some of these country names appear to be aggregations and not actual countries. There are geographic agreggations, aggregations based on income and even political/geo-political agregations, such as the Arab League or the Euro area. In order to keep this column strictly with countries, let's try to remove the ones which are present somehow on the region column and keep the ones which have no match.

In [163]:
regions = pd.DataFrame(dataset_2021['region'].drop_duplicates().dropna(), columns=['region'])

In [164]:
odd_countries = pd.DataFrame(dataset_2021.loc[dataset_2021['region'].isna(), 'country_name'].drop_duplicates())

In [165]:
odd_countries['match'] = odd_countries['country_name']\
    .apply(lambda x: regions[regions['region'].apply(lambda y: y in x)]['region'].max())

We will also remove anything related to income, as we will later perform these aggregations.

In [166]:
odd_countries.loc[odd_countries['country_name'].str.contains('income', flags=re.IGNORECASE),'match'] = 'Income'

Let's drop everything with no match so far and see what we are left with.

In [167]:
odd_countries = odd_countries[odd_countries['match'].isna()]

In [168]:
odd_countries

,country_name,match
0,Africa Eastern and Southern,NaN
443,Africa Western and Central,NaN
886,Arab World,NaN
1329,Caribbean small states,NaN
1772,Central Europe and the Baltics,NaN
2215,Early-demographic dividend,NaN
3987,Euro area,NaN
5759,European Union,NaN
6202,Fragile and conflict affected situations,NaN
6645,Heavily indebted poor countries (HIPC),NaN


We still have many countries without any match. There are some patterns we can use, but we will have to use the information provided with the dataset to manually filter some of them and flag them as to be removed.

In [175]:
odd_countries.loc[odd_countries['country_name'].str.contains('IDA|IBRD', regex=True, flags=re.IGNORECASE),'match'] = 'Remove'

In [176]:
odd_countries

,country_name,match
0,Africa Eastern and Southern,NaN
443,Africa Western and Central,NaN
886,Arab World,NaN
1329,Caribbean small states,NaN
1772,Central Europe and the Baltics,NaN
2215,Early-demographic dividend,NaN
3987,Euro area,NaN
5759,European Union,NaN
6202,Fragile and conflict affected situations,NaN
6645,Heavily indebted poor countries (HIPC),NaN


In [177]:
odd_countries.loc[odd_countries['country_name'].str.contains('dividend', flags=re.IGNORECASE),'match'] = 'Remove'

In [184]:
list_to_remove = ['Arab World','Euro area','European Union','OECD members','Small states','Other small states','Least developed countries: UN classification'] #Geopolitical aggregations
list_to_remove.extend(['Fragile and conflict affected situations','Heavily indebted poor countries (HIPC)'
,'Not classified','World']) #Situational and others

In [185]:
odd_countries.loc[odd_countries['country_name'].isin(list_to_remove),'match'] = 'Remove'

In [187]:
odd_countries = odd_countries[odd_countries['match'].isna()]

In [188]:
odd_countries

,country_name,match
0,Africa Eastern and Southern,NaN
443,Africa Western and Central,NaN
1329,Caribbean small states,NaN
1772,Central Europe and the Baltics,NaN
16834,Pacific island small states,NaN


Quite skeptical about the Central Europe and the Baltics aggregation, it feels like we would have enough data for it to be a region, but since we have no proof other wise, it will stay here.

In [196]:
dataset_2021_region_clean = dataset_2021\
    .loc[ (~dataset_2021['region'].isna())
        |( (dataset_2021['region'].isna() ) & (dataset_2021['country_name'].isin(odd_countries['country_name'])) ) ]

In [197]:
dataset_2021_region_clean.loc[dataset_2021_region_clean['region'].isna(),'region'] =  dataset_2021_region_clean['country_name']